In [198]:
import pandas as pd

In [199]:
df = pd.read_csv('scrapedResults.csv')

## Data cleaning

In [200]:
#---Precio column---#
# Delete everything after the first whitespace in the 'precio' column
df['precio'] = df['precio'].str.split().str[0]
# Remove empty spaces from the precio column
df['precio'] = df['precio'].str.strip()
# Remove the commas in the precio column
df['precio'] = df['precio'].str.replace(',', '')
# Set the data type of the 'precio' column to float, this is used to fill the missing values with the mean of the column
df['precio'] = df['precio'].astype(float)

# # Forward-fill missing values within each group (car model) 
df['precio'] = df.groupby('modelo')['precio'].fillna(method='ffill')
# Forward-fill missing values within each group (car brand)
df['precio'] = df.groupby('marca')['precio'].fillna(method='ffill')
# Fill the remaining missing values with the mean of the column
df['precio'] = df['precio'].fillna(df['precio'].mean())

#---Renimiento column---#
# Convert the column to numeric, replacing non-numeric values with NaN
df['rendimiento'] = pd.to_numeric(df['rendimiento'], errors='coerce')
# Calculate the mean of the numeric values in the column, excluding NaN values
mean_value = df['rendimiento'].mean(skipna=True)
# Fill the NaN and non-numeric values in the column with the mean value
df['rendimiento'].fillna(mean_value, inplace=True)

#---Transmisión column---#
# Repeat the process for other columns
df['transmisión'] = df.groupby('modelo')['transmisión'].fillna(method='ffill')
df['transmisión'] = df.groupby('marca')['transmisión'].fillna(method='ffill')

#---Combustible column---#
# Drop rows with numbers in the combustible column
df = df[~df['combustible'].str.contains(r'\d', na=False)]
# Fill missing cells in 'combustible' column with values from similar rows
df['combustible'] = df.groupby(['marca', 'modelo', 'año'])['combustible'].fillna(method='ffill')
# If there are still missing values, fill them with values from the same 'marca'
df['combustible'] = df.groupby(['marca'])['combustible'].fillna(method='ffill')
# If there are still missing values, fill them with values from the same 'modelo'
df['combustible'] = df.groupby(['modelo'])['combustible'].fillna(method='ffill')
# If there are still missing values, fill them with the most common value in the column
df['combustible'] = df['combustible'].fillna(df['combustible'].mode()[0])

#---Litros column---#
# Repeat the process for other columns
df['litros'] = df.groupby(['marca', 'modelo', 'año'])['litros'].fillna(method='ffill')
df['litros'] = df.groupby('modelo')['litros'].fillna(method='ffill')
df['litros'] = df.groupby('marca')['litros'].fillna(method='ffill')
df['litros'] = df['litros'].fillna(df['litros'].mode()[0])

# Print if there are any missing values left
print(df.isnull().sum())

marca          0
modelo         0
año            0
precio         0
transmisión    0
rendimiento    0
combustible    0
litros         0
dtype: int64


In [201]:
# Save the cleaned data to a new csv file
df.to_csv('cleanedScrapedResults.csv', index=False)